```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

### Testing playground for different boundary and initial conditions

This notebook is used mainly as a playground for testing and different ideas, and results or tests that are ment for significant are moved to individual notebooks.

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

import os
import pyopencl
import datetime
import sys

# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, RecursiveCDKLM16, DataOutput, SimWriter, PlotHelper, Common
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [ ]:
if 'sim' in globals():
    print "Found sim, so we will clean up!"
    sim.cleanUp()
    print " success :)"
else:
    print "Did not find sim in globals, and that's fine as well!"

reload(Common)
reload(CDKLM16)
    
#Coriolis well balanced reconstruction scheme  - NUMERICAL SPONGE TEST
nx = 10
ny = 20

dx = 200.0
dy = 200.0

dt = 0.95/10.0
g = 9.81

f = 0.00
r = 0.0
wind = Common.WindStressParams(type=99)


bcSettings = 5
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
elif bcSettings == 4:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
elif bcSettings == 5:
    # Numerical Sponge
    ghosts = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=ghosts)
    validDomain = [None, None, 0, 0]
    

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
#print(dataShape)

print ("dataShape from notebook: ", dataShape)

waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');
#print(v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]].shape)

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)

addCentralBump(h0, nx, ny, dx, dy, ghosts)
for x in range(nx + ghosts[0] + ghosts[2]):
    for y in range(ny + ghosts[1] + ghosts[3]):
        if (x == 10 or x == nx + 9) and (y > 10 and y < ny + 9):
            h0[y,x] += 1
        if (y == 10 or y == ny + 9) and (x >= 10 and x <= nx + 9):
            h0[y,x] += 1


fig = plt.figure(figsize=(3,3))
#plt.imshow(h0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], interpolation="None")
plt.imshow(h0, interpolation="None")
plt.colorbar()

#Initialize simulator
reload(CDKLM16)
reload(KP07)
reload(Common)
#sim = KP07.KP07(cl_ctx, \
sim = CDKLM16.CDKLM16(cl_ctx, \
                h0, u0, v0, \
                Bi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions)

t = sim.step(0.0)
h1, u1, v1 = sim.download()
fig = plt.figure(figsize=(3,3))
plt.imshow(h1, interpolation="None")
plt.colorbar()

In [ ]:
#Calculate radius from center of bump for plotting
x_center = dx*(nx+20)/2.0
y_center = dy*(ny+20)/2.0
y_coords, x_coords = np.mgrid[0:(ny+20)*dy:dy, 0:(nx+20)*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                (h1 - waterHeight)/30, u1, v1)

In [ ]:
nx = 100
ny = 200

dx = 200.0
dy = 200.0
ghosts = [3,3,3,3]

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

In [ ]:
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.ones(dataShape, dtype=np.float32, order='C')
v0 = np.ones(dataShape, dtype=np.float32, order='C')

addDualVortex(h0, u0, v0, nx, ny, dx, dy, ghosts)

#singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, u0, interpolation_type="None")
fig = plt.figure()
plt.imshow(u0, interpolation="None")
#singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, u0, interpolation_type="None")
fig = plt.figure()
plt.plot(h0[:, 53])
print(h0.shape)

In [ ]:
ghosts = [3,3,3,3]

B = np.zeros((ny+ghosts[0]+ghosts[2], nx+ghosts[1]+ghosts[3]), dtype=np.float32);
#makeBathymetryCrazyness(h0, nx, ny, dx, dy, ghosts)
#linearBathymetryY(B, nx, ny, dx, dy, ghosts, 5, 30)
linearBathymetryX(B, nx, ny, dx, dy, ghosts, 5, 50)
#diagonalWallBathymetry(B, nx, ny, dx, dy, ghosts, 50)
reload(PlotHelper)
fig = plt.figure()
#singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, B, interpolation_type="None")
plt.imshow(B, interpolation="None")
print(h0.shape)
h0shapeX, h0shapeY = h0.shape
print(np.min(h0), np.max(h0))
print(range(0,2))
fig = plt.figure()
plt.plot(B[82, :])


In [ ]:
# Forward backward linear

#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
f = 0.0
r = 0.0

#wind = Common.WindStressParams(type=0, tau0=0.1, rho=1025, alpha=0.000005)
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)


bcSettings = 1
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
    validDomain = [None, None, 0, 0]
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] # Both periodic
    validDomain = [-1, -1, 0, 0]
elif bcSettings == 3:
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
    ghosts = [1,0,0,0] # periodic north-south
    validDomain = [-1, None, 0, 0]
else:
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    ghosts = [0,1,0,0] # periodic east-west
    validDomain = [None, -1, 0, 0]


h0 = np.ones((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32) * 60;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32);
u0 = np.zeros((ny+ghosts[0], nx+1), dtype=np.float32);
v0 = np.zeros((ny+1, nx+ghosts[1]), dtype=np.float32);

#Create bump in to lower left of domain for testing
addCentralBump(eta0, nx, ny, dx, dy, ghosts)


#Initialize simulator
reload(FBL)
sim = FBL.FBL(cl_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r, \
              wind_stress=wind, \
              boundary_conditions=boundaryConditions, \
              write_netcdf=True)


#Calculate radius from center of bump for plotting
x_center = dx*nx*0.3
y_center = dy*ny*0.2
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))
   
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]],\
                                u0[validDomain[2]:validDomain[0], :], \
                                v0[:, validDomain[3]:validDomain[1]])
T = 20
def animate(i):
    if (i>0):
        #t = sim.step(1.0)
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]],\
                 u1[validDomain[2]:validDomain[0], :], \
                 v1[:, validDomain[3]:validDomain[1]]);

    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_fbl.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
sim.cleanUp()

In [ ]:
#Centered in time, centered in space

#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()

nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
r = 0.0
A = 1
f = 0.01

#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)

    

bcSettings = 1
ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # periodic north-south
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # periodic east-west
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 60;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+1+2, nx+2), dtype=np.float32, order='C');

#Create bump in to lower left of domain for testing
x_center = dx*nx*0.3
y_center = dy*ny*0.2
#makeUpperCornerBump(eta0, nx, ny, dx, dy, ghosts)
#makeCentralBump(eta0, nx, ny, dx, dy, ghosts)
#makeCornerBump(eta0, nx, ny, dx, dy, ghosts)
#makeLowerLeftBump(eta0, nx, ny, dx, dy, ghosts)
#addDualVortex(eta0, u0, v0, nx, ny, dx, dy, ghosts)
addDualVortexStaggered(eta0, u0, v0, nx, ny, dx, dy, ghosts)

    

#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, \
                wind_stress=wind, \
                write_netcdf=True, \
                boundary_conditions=boundaryConditions )


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

ghosts = [-1,1,-1,1]

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
sim.cleanUp()

In [ ]:
a,b,c = sim.download()

In [ ]:
if 'sim' in globals():
    print "Found sim, so we will clean up!"
    sim.cleanUp()
    print " success :)"
else:
    print "Did not find sim in globals, and that's fine as well!"

    
#Coriolis well balanced reconstruction scheme
nx = 30
ny = 60

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81

f = 0.00
r = 0.0


bcSettings = 5
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([-2,-2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
elif bcSettings == 4:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
elif bcSettings == 5:
    # Numerical Sponge
    ghosts = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=ghosts)
    validDomain = [None, None, 0, 0]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');
#print(v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]].shape)

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)

#addCentralBump(h0, nx, ny, dx, dy, ghosts)
addLowerLeftBump(h0, nx, ny, dx, dy, ghosts)

# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)
f = 0.0
#f = 0.02

#Initialize simulator
reload(CDKLM16)
reload(Common)
reload(RecursiveCDKLM16)
sim = CDKLM16.CDKLM16(cl_ctx, \
#sim = RecursiveCDKLM16.RecursiveCDKLM16(cl_ctx, \
                h0, u0, v0, \
                Bi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions, \
                reportGeostrophicEquilibrium=True,
                write_netcdf=True)

#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
print ("before if: ", y_coords.shape, x_coords.shape)
if bcSettings == 5:
    x_center = dx*(nx+20)/2.0
    y_center = dy*(ny+20)/2.0
    y_coords, x_coords = np.mgrid[0:(ny+20)*dy:dy, 0:(nx+20)*dx:dx]
    print ("inside if: ", y_coords.shape, x_coords.shape)
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                                v0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]])

#T = 200
T = 40
Kx = np.zeros(T)
Ly = np.zeros(T)
uxpvy = np.zeros(T)

def animate(i):
    if (i>0):
        #t = sim.step(10.0)
        t = sim.step(10.0)
        #t = sim.step(250.0)
        #t = sim.step(100.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()
    uxpvy[i], Kx[i], Ly[i] = sim.downloadGeoEqNorm()
    #print uxpvy[i], Kx[i], Ly[i]
    
    brighten = 10
    
    plotter.plot(brighten*(h1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight), 
                 brighten*u1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                 brighten*v1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\t(Min, max) h: " + str((np.min(h1),np.max(h1))) + \
        "\tMax (u, v): " + str((np.max(u1), np.max(v1)))
        
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
sim.cleanUp()

In [ ]:
a,b,c = sim.download()
a.shape

In [ ]:
#Coriolis well balanced reconstruction scheme

if 'sim' in globals():
    sim.cleanUp()

nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/10.0
g = 9.81

f = 0.00
r = 0.0


bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
#print(dataShape)


waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');
#print(v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]].shape)

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
straightWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)
#makeBathymetryCrazyness(Bi, nx+1, ny+1, dx, dy, ghosts)
#linearBathymetryX(Bi, nx, ny, dx, dy, ghosts, 5, 50)
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
#addCentralBump(h0, nx, ny, dx, dy, validDomain)
#addLowerLeftBump(h0, nx, ny, dx, dy, validDomain)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, validDomain)
addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)  
#addDualVortex(h0, u0, v0, nx, ny, dx, dy, validDomain)
#addCentralBump(h0, nx, ny, dx, dy, validDomain)

# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)
f = 0.01

#Initialize simulator
reload(CDKLM16)
reload(Common)
reload(RecursiveCDKLM16)
sim = CDKLM16.CDKLM16(cl_ctx, \
#sim = RecursiveCDKLM16.RecursiveCDKLM16(cl_ctx, \
                h0, u0, v0, \
                Bi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions, \
                reportGeostrophicEquilibrium=True)

#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

#T = 200
T = 50
Kx = np.zeros(T)
Ly = np.zeros(T)
uxpvy = np.zeros(T)

def animate(i):
    if (i>0):
        t = sim.step(10.0)
        #t = sim.step(250.0)
        #t = sim.step(100.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()
    uxpvy[i], Kx[i], Ly[i] = sim.downloadGeoEqNorm()
    #print uxpvy[i], Kx[i], Ly[i]
    
    brighten = 1
    
    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#* Example of plotting with NetCDF referance *#

reload(SimWriter)
reload(Common)
a = SimWriter.SimNetCDFWriter(sim)
print a

fig = plt.figure()
gs = gridspec.GridSpec(2,1, height_ratios=[3.5, 1])

fig.add_subplot(gs[0])
plt.plot(uxpvy, 'b', label="ux+vy")
plt.plot(Kx, 'r', label="Kx")
plt.plot(Ly, 'g', label="Ly")
plt.legend()

ax = fig.add_subplot(gs[1])
a.infoPlot(ax)
plt.show()

a.__exit__(0,0,0)

In [ ]:
h3, hu3, hv3 = sim.download()
print("Initial water volume: ", sum(sum(h0)))
print("After simulation:     ", sum(sum(h3)))
print(imgdir)
print("min-max h:            ", np.min(h3), np.max(h3))
print("min-max hu:           ", np.min(hu3), np.max(hu3))
print("min-max hv:           ", np.min(hv3), np.max(hv3))
print(h3.shape)
plt.figure()
plt.subplot(211)
plt.plot(hu3[100, :], 'b', label="hu")
plt.plot(hv3[100, :], 'r', label="hv")
plt.legend()
plt.title("Momentum on y = 100*dy")
plt.subplot(212)
plt.plot(hu3[:, 50], 'b', label="hu")
plt.plot(hv3[:, 50], 'r', label="hv")
plt.legend()
plt.title("Momentum on x = 50*dx")

fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, np.sqrt(hu3*hu3 + hv3*hv3), interpolation_type="None")


In [ ]:
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - 1.0, 
                                u0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                                v0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]])
#h1, u1, v1 = sim.download()
#fig = plt.figure()
#singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, h1)
#fig = plt.figure()
#singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, v1)


In [ ]:
h1, u1, v1 = sim.download()
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius,
                 h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);

print(sum(sum(h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])))
print(sum(sum(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])))
print(60*nx*ny)
print(7./20000/500)
print(imgdir)

print("---- Finding largest eigenvalue ---- ")
a = np.array([[1,2], [3,4]])
b = np.array([[2,1], [4,3]])
c = np.abs(np.array([[2,10], [-40,3]]))
print(np.max(np.maximum(a,np.maximum(b,c))))

h = h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]
u = u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] 
v = v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]

lum = np.max(np.abs(u - np.sqrt(g*h)))
lup = np.max(np.abs(u + np.sqrt(g*h)))
lvm = np.max(np.abs(v - np.sqrt(g*h)))
lvp = np.max(np.abs(v + np.sqrt(g*h)))

eigmax = max( lum, lup, lvm, lvp)

0.25*dx/eigmax

In [ ]:
h1, u1, v1 = sim.download()
#print(e)
print(max(h1[:,4]))
print(max(h1[4,:]))
print("h1.shape", h1.shape)
print("(ny, nx)", (ny, nx))
print("h1[4,:].shape", h1[4,:].shape)

i = 3
print("np.linalg.norm(h1[:,i] - h1[:,nx+i])", np.linalg.norm(h1[:,i] - h1[:,nx+i]))
plt.plot(h1[:,i], 'bd')
plt.plot(h1[:, i+nx], 'r')
plt.plot(h1[i,:], 'gd')
plt.plot(h1[i+ny, :], 'k')
waterHeight
#print(np.linalg.norm(h1[,:] - h1[ny+10, :]))

In [ ]:
# Kurganov-Petrova 2007 paper
reload(KP07)
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt = 0.95/100
g = 9.81

f = 0.00
r = 0.0

bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#makeBathymetryCrater(Bi, nx+1, ny+1, dx, dy, ghosts)
makeBathymetryCrazyness(Bi, nx+1, ny+1, dx, dy, ghosts)
#linearBathymetryX(Bi, nx, ny, dx, dy, ghosts, 5, 50)
#diagonalWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)
#straightWallBathymetry(Bi, nx, ny, dx, dy, ghosts, 50)

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
#addCentralBump(h0, nx, ny, dx, dy, validDomain)
#addLowerLeftBump(h0, nx, ny, dx, dy, validDomain)
#addDebugBump(h0, nx, ny, dx, dy, 0.2, 0.1, validDomain)
#addDiagonalDam(h0, nx, ny, dx, dy, ghosts, 0.05)
#addStraightDam(h0, nx, ny, dx, dy, ghosts, 0.1)


# WIND
#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)
f = 0.00

#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                h0, Bi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions,
                use_rk2=True,
                write_netcdf=True)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] )

T = 100
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] );
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1))
        #fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
sim.cleanUp()

In [ ]:
h3, hu3, hv3 = sim.download()
print("Initial water volume: ", sum(sum(h0)))
print("After simulation:     ", sum(sum(h3)))
print(imgdir)
print("min-max h:           ", np.min(h3), np.max(h3))
print("min-max hu:           ", np.min(hu3), np.max(hu3))
print("min-max hv:           ", np.min(hv3), np.max(hv3))

In [ ]:
# Quickly testing initBm
Bi2, Bm2 = sim.downloadBathymetry()

y_coords, x_coords = np.mgrid[0:(ny+1)*dy:dy, 0:(nx+1)*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)

fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, Bi2)

y_coords, x_coords = np.mgrid[0:(ny)*dy:dy, 0:(nx)*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)

fig = plt.figure()
singlePlotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, Bm2)

print(np.max(Bi2))
print(np.max(Bm2))

In [ ]:
plt.plot(Bm2[ 50, :])
print(Bm2.shape)
print(np.max(Bm2), np.min(Bm2))
print(Bm2[50, :])

In [ ]:
a = [1,2,3,4]
a[1]
eta1, hu1, hv1 = sim.download()
print(eta1.max())
print(eta1.min())
print(hu1.max())
print(hu1.min())
print(hv1.max())
print(hv1.min())
#eta0 = np.zeros((5,4), dtype=np.float32);
#print(eta0)
#print(eta0.shape)
#print(eta0[0:None,
#           1:-0])
print(ghosts)
if (ghosts[0] == None):
    ghosts[0] = 0

for i in range(0, nx+ghosts[1]-1) :
    for j in range(0, ny+ghosts[0] -1):
        if eta1[j,i] != eta1[j,i]:
            print("i: " + str(i) + " and j: " + str(j))




#eta0 = np.zeros((ny+ghosts[0], nx+ghosts[1])

print(nx)
print(dx)
x = np.linspace(dx/2,(nx-0.5)*dx, nx)
print(x.shape)
#print(x)
print(np.linspace(0,9,10))

B = lambda: None
setattr(B, 'h', "hei") 
hasattr(B, 'h')
B.h

ghosts*2-5